In [1]:
import polars as pl
import gzip
from tqdm import tqdm

In [ ]:
# You can download the dataset here
# https://www.kaggle.com/datasets/amananandrai/ag-news-classification-dataset

In [2]:
df = pl.read_csv("train.csv")
df.shape

(120000, 3)

In [3]:
df.groupby(
    pl.col("Class Index")
).count()

Class Index,count
i64,u32
2,30000
1,30000
4,30000
3,30000


In [4]:
df_test = pl.read_csv("test.csv")
df_test.head()

Class Index,Title,Description
i64,str,str
3,"""Fears for T N …","""Unions represe…"
4,"""The Race is On…","""SPACE.com - TO…"
4,"""Ky. Company Wi…","""AP - A company…"
4,"""Prediction Uni…","""AP - It's bare…"
4,"""Calif. Aims to…","""AP - Southern …"


In [5]:
train_set = df.select("Description", "Class Index")
test_set = df_test.select("Description", "Class Index")

In [7]:
test = test_set.with_columns(
    compressed = pl.col('Description').apply(str.encode).apply(gzip.compress)
).select(
        "Description"
        , "compressed"
        , pl.col("compressed").apply(len).alias("compressed_len")
        , "Class Index"
)

In [8]:
train = train_set.with_columns(
    compressed = pl.col('Description').apply(str.encode).apply(gzip.compress)
).select(
        "Description"
        , "compressed"
        , pl.col("compressed").apply(len).alias("compressed_len")
        , "Class Index"
)

In [9]:
def compressed_knn_vote(text:str, c_length:int, train:pl.DataFrame) -> int:
    return train.lazy().with_columns(
        ncd = (
            (   
                pl.col("Description") + pl.lit(text)).apply(str.encode).apply(gzip.compress).apply(len) 
                - pl.min_horizontal(c_length, pl.col("compressed_len"))
            ) 
            / pl.max_horizontal(c_length, pl.col("compressed_len")
        )
    ).filter(
        pl.col("ncd").arg_sort() < 3
    ).select(
        pl.col("Class Index").mode().first()
    ).collect().item(0,0)

In [10]:
# Too slow...

predictions = []
for des, l1 in tqdm(zip(test["Description"], test["compressed_len"]), total=len(test)):
    predictions.append(
        compressed_knn_vote(des, l1, train)
    )

  5%|▍         | 361/7600 [12:38<4:13:24,  2.10s/it]


KeyboardInterrupt: 

In [ ]:
# # Still slow.
# import os
# from concurrent.futures import ThreadPoolExecutor, as_completed

# predictions = []
# with ThreadPoolExecutor(max_workers=os.cpu_count()) as ex:
#     pbar = tqdm(total=len(test))
#     for future in as_completed(ex.submit(compressed_knn_vote, des, l1, train) 
#                                 for des, l1 in zip(test["Description"], test["compressed_len"])):
        
#         predictions.append(future.result())
#         pbar.update(1)

#     pbar.close()


In [ ]:
# Original Code
# I cannot get this working. It is unclear what top_k_class is, and how this has a count method.

# import gzip
# import numpy as np

# k = 3
# x1: str
# for x1,_ in test_set:
#     Cx1 = len(gzip.compress(x1.encode()))
#     distance_from_x1 = []
#     x2: str
#     for x2,_ in train_set:
#         Cx2 = len(gzip.compress(x2.encode()))
#         x1x2 = " ".join([x1,x2])
#         Cx1x2 = len(gzip.compress(x1x2.encode()))
#         ncd = (Cx1x2 - min(Cx1,Cx2)) / max (Cx1,Cx2)
#         distance_from_x1.append(ncd)
        
#     sorted_idx = np.argsort(np.array(distance_from_x1))
#     top_k_class = train_set[sorted_idx[:k],1]

#     predict_class = max(set(top_k_class), key = top_k_class.count)